<a href="https://colab.research.google.com/github/thedavidneufeld/ResearchCode/blob/main/RQI_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports**

In [1]:
import numpy as np

**Helper Functions**

In [2]:
# given an approximate eigenvalue (mu) and a matrix M
# returns (mu)*I-M
def A(mu, M):
    I = np.eye(M.shape[0])
    return mu*I-M

# calculates and returns the conditon number of M
# based on sigma_1/sigma_n
def svd_cond(M):
    SVD = np.linalg.svd(M)
    return SVD[1][0]/SVD[1][-1]

# calculates and returns sigma_1/sigma_{n-1}
def svd_desired_cond(M):
    SVD = np.linalg.svd(M)
    return SVD[1][0]/SVD[1][-2]

# calculates a preconditioned matrix C based on
# an approximate eigenvalue (mu), a matrix M, and vectors
# y and v
# returns C
def f(mu, M, y, v):
    # convert y and v to column vectors
    Y = y[:, np.newaxis]
    V = v[:, np.newaxis]
    # calculate (mu)*I-M
    A1 = A(mu, M)
    # calculate ((mu)*I-M)/||((mu)*I-M)||+YV^H
    C = A1/np.linalg.norm(A1, 2) + np.dot(Y, V.T.conj())
    # return C
    return C

**RQI Algorithm**

In [3]:
# based on a close approximation of an eigenvalue of M (mu)
# returns the corresponding eigenvector as well as the correct eigenvalue
# takes in a matrix M, eigenvalue approximation mu
# tolerance ep, and a max number of iterations it
def RQI(M, mu, ep, it):
    # create a random vector with norm of 1
    # vector needs to be compatible with M
    y = np.random.rand(M.shape[0])
    y = y/np.linalg.norm(y)
    # track condition number of (mu)I-M through iterations
    conds = np.empty(0)
    # repeat until stop condition is met
    # or until the maximum number of iterations is reached
    for i in range(1, it+1):
        # calculate (mu)I-M
        muI_M = A(mu, M)
        # calculate the condition number of (mu)I-M
        conds = np.append(conds, svd_cond(muI_M))
        # solve the system ((mu)I-M)x=y
        x = np.linalg.solve(muI_M, y)
        # update y with a normalized x
        y = x/np.linalg.norm(x)
        # update mu with (x^T)Mx
        mu = np.dot(np.dot(y.T, M), y)
        # if ||Mx-(mu)x|| <= ep*||M||, then return mu, y, i, and conds
        if np.linalg.norm(np.dot(M, x)-mu*x, 2) <= ep*np.linalg.norm(M, 2):
            return mu, y, i, conds
    # if max iterations has been surpassed, return None values,
    # the number of iterations, and conds
    return None, None, it, conds


**RQI Algorithm with Additive Preconditioning**

In [4]:
# based on a close approximation of an eigenvalue of M (mu)
# returns the corresponding eigenvector as well as the correct eigenvalue
# takes in a matrix M, eigenvalue approximation mu
# tolerance ep, and a max number of iterations it
#
# similar to regular RQI, but uses additive preprocessing
def RQIAPP(M, mu, ep, it):
    # create two random vectors with norm of 1
    # vectors need to be compatible with M
    y = np.random.rand(M.shape[0])
    y = y/np.linalg.norm(y)
    v = np.random.rand(M.shape[0])
    v = v/np.linalg.norm(v)
    # track condition number of (mu)I-M, C, and the desired condition number
    conds = np.empty(0)
    conds_C = np.empty(0)
    conds_desired = np.empty(0)
    # repeat until stop condition is met
    # or until the maximum number of iterations is reached
    for i in range(1, it+1):
        # calculate (mu)I-M
        muI_M = A(mu, M)
        # calculate C
        C = f(mu, M, y, v)
        # calculate the condition numbers
        conds = np.append(conds, svd_cond(muI_M))
        conds_C = np.append(conds_C, svd_cond(C))
        conds_desired = np.append(conds_desired, svd_desired_cond(muI_M))
        # solve the system Cx=y
        x = np.linalg.solve(C, y)
        # update y with a normalized x
        y = x/np.linalg.norm(x)
        # update mu with (y^T)My
        mu = np.dot(np.dot(y.T, M), y)
        # if ||Mx-(mu)x|| <= ep*||M||, then return mu, y, i, and conds
        if np.linalg.norm(np.dot(M, x)-mu*x, 2) <= ep*np.linalg.norm(M, 2):
            return mu, y, i, conds, conds_C, conds_desired
    # if max iterations has been surpassed, return None values,
    # the number of iterations, and conds
    return None, None, it, conds, conds_C, conds_desired

**Test and Compare Algorithms**

In [5]:
# Setup

# create a random matrix M
M = np.random.rand(100, 100)
# calculate the eigenvalues/eigenvectors of M
vals, vecs = np.linalg.eig(M)
# extract the first eigenvalue of M and subtract
# a small value to make it an approximation
mu = vals[0]-0.25

In [6]:
# Print condition number of M
svd_cond(M)

1881.81440458731

In [7]:
# Test RQI with 1e-4
result = RQI(M, mu, 1e-4, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])

Eigenvalue: (49.83826642956268+0j)
Eigenvector: [0.09550871+0.j 0.10338972+0.j 0.09767328+0.j 0.09968595+0.j
 0.10344816+0.j 0.10123636+0.j 0.09808322+0.j 0.10422956+0.j
 0.10085886+0.j 0.09693695+0.j 0.10004665+0.j 0.09993394+0.j
 0.10008715+0.j 0.09407992+0.j 0.10323379+0.j 0.09513032+0.j
 0.09957522+0.j 0.09707391+0.j 0.09453099+0.j 0.10903108+0.j
 0.0890976 +0.j 0.10702368+0.j 0.10955024+0.j 0.09263746+0.j
 0.11312551+0.j 0.09863057+0.j 0.10169131+0.j 0.1039357 +0.j
 0.0873236 +0.j 0.09470451+0.j 0.09873802+0.j 0.10854135+0.j
 0.10494186+0.j 0.10126689+0.j 0.10181169+0.j 0.09940483+0.j
 0.09636203+0.j 0.09995023+0.j 0.09126764+0.j 0.11414029+0.j
 0.10043883+0.j 0.10341104+0.j 0.09581323+0.j 0.09970896+0.j
 0.09818493+0.j 0.09816776+0.j 0.09204125+0.j 0.10046363+0.j
 0.10267228+0.j 0.09303735+0.j 0.10462168+0.j 0.10536937+0.j
 0.10263202+0.j 0.10174593+0.j 0.10459266+0.j 0.09712412+0.j
 0.09947651+0.j 0.09179901+0.j 0.10769462+0.j 0.10912235+0.j
 0.09876485+0.j 0.10464538+0.j 0.0962

In [8]:
# Test RQIAPP with 1e-4
result = RQIAPP(M, mu, 1e-4, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers for (mu)I-M:', result[3])
print('Condition Numbers for C:', result[4])
print('Desired Condition Numbers:', result[5])

Eigenvalue: (49.83826642714531+0j)
Eigenvector: [0.09550871+0.j 0.10338972+0.j 0.09767328+0.j 0.09968595+0.j
 0.10344816+0.j 0.10123637+0.j 0.09808322+0.j 0.10422956+0.j
 0.10085886+0.j 0.09693695+0.j 0.10004665+0.j 0.09993394+0.j
 0.10008716+0.j 0.09407992+0.j 0.10323379+0.j 0.09513032+0.j
 0.09957521+0.j 0.09707391+0.j 0.094531  +0.j 0.10903108+0.j
 0.0890976 +0.j 0.10702367+0.j 0.10955025+0.j 0.09263746+0.j
 0.11312551+0.j 0.09863057+0.j 0.10169132+0.j 0.1039357 +0.j
 0.08732361+0.j 0.09470451+0.j 0.09873801+0.j 0.10854135+0.j
 0.10494186+0.j 0.10126689+0.j 0.10181169+0.j 0.09940484+0.j
 0.09636202+0.j 0.09995023+0.j 0.09126765+0.j 0.1141403 +0.j
 0.10043883+0.j 0.10341103+0.j 0.09581322+0.j 0.09970896+0.j
 0.09818492+0.j 0.09816775+0.j 0.09204125+0.j 0.10046363+0.j
 0.10267228+0.j 0.09303735+0.j 0.10462168+0.j 0.10536937+0.j
 0.10263202+0.j 0.10174593+0.j 0.10459266+0.j 0.09712412+0.j
 0.09947651+0.j 0.091799  +0.j 0.10769462+0.j 0.10912235+0.j
 0.09876486+0.j 0.10464538+0.j 0.0962

In [9]:
# Test RQI with 1e-6
result = RQI(M, mu, 1e-6, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])

Eigenvalue: (49.838266428418+0j)
Eigenvector: [0.09550871+0.j 0.10338972+0.j 0.09767328+0.j 0.09968594+0.j
 0.10344816+0.j 0.10123636+0.j 0.09808322+0.j 0.10422956+0.j
 0.10085886+0.j 0.09693695+0.j 0.10004665+0.j 0.09993394+0.j
 0.10008715+0.j 0.09407992+0.j 0.10323378+0.j 0.09513031+0.j
 0.09957521+0.j 0.09707391+0.j 0.09453099+0.j 0.10903108+0.j
 0.0890976 +0.j 0.10702368+0.j 0.10955025+0.j 0.09263746+0.j
 0.11312551+0.j 0.09863057+0.j 0.10169131+0.j 0.1039357 +0.j
 0.08732361+0.j 0.09470451+0.j 0.09873801+0.j 0.10854135+0.j
 0.10494186+0.j 0.10126689+0.j 0.10181169+0.j 0.09940484+0.j
 0.09636202+0.j 0.09995023+0.j 0.09126765+0.j 0.1141403 +0.j
 0.10043883+0.j 0.10341103+0.j 0.09581322+0.j 0.09970896+0.j
 0.09818492+0.j 0.09816775+0.j 0.09204125+0.j 0.10046363+0.j
 0.10267228+0.j 0.09303735+0.j 0.10462168+0.j 0.10536937+0.j
 0.10263202+0.j 0.10174593+0.j 0.10459266+0.j 0.09712412+0.j
 0.09947651+0.j 0.091799  +0.j 0.10769462+0.j 0.10912235+0.j
 0.09876486+0.j 0.10464538+0.j 0.096220

In [10]:
# Test RQIAPP with 1e-6
result = RQIAPP(M, mu, 1e-6, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])
print('Condition Numbers for C:', result[4])
print('Desired Condition Numbers:', result[5])

Eigenvalue: (49.83826644519515+0j)
Eigenvector: [0.09550872+0.j 0.10338971+0.j 0.09767328+0.j 0.09968594+0.j
 0.10344816+0.j 0.10123636+0.j 0.09808321+0.j 0.10422955+0.j
 0.10085886+0.j 0.09693696+0.j 0.10004664+0.j 0.09993394+0.j
 0.10008717+0.j 0.09407992+0.j 0.10323377+0.j 0.09513032+0.j
 0.09957522+0.j 0.09707392+0.j 0.09453099+0.j 0.10903109+0.j
 0.08909761+0.j 0.10702368+0.j 0.10955024+0.j 0.09263747+0.j
 0.11312551+0.j 0.09863056+0.j 0.10169131+0.j 0.10393569+0.j
 0.08732361+0.j 0.09470451+0.j 0.09873801+0.j 0.10854136+0.j
 0.10494185+0.j 0.1012669 +0.j 0.10181168+0.j 0.09940483+0.j
 0.09636202+0.j 0.09995023+0.j 0.09126764+0.j 0.11414029+0.j
 0.10043883+0.j 0.10341104+0.j 0.09581323+0.j 0.09970896+0.j
 0.09818492+0.j 0.09816774+0.j 0.09204125+0.j 0.10046362+0.j
 0.10267228+0.j 0.09303736+0.j 0.10462169+0.j 0.10536937+0.j
 0.10263203+0.j 0.10174594+0.j 0.10459266+0.j 0.09712411+0.j
 0.0994765 +0.j 0.09179901+0.j 0.10769463+0.j 0.10912234+0.j
 0.09876485+0.j 0.10464538+0.j 0.0962

In [11]:
# Test RQI with 1e-8
result = RQI(M, mu, 1e-8, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])

Eigenvalue: None
Eigenvector: None
Iterations: 100
Condition Numbers: [2.14866173e+02 2.78617320e+04 9.23081484e+08 1.69422920e+15
 2.93344489e+15 4.99494454e+15 4.99494454e+15 1.20947758e+16
 1.20947758e+16 1.20947758e+16 1.20947758e+16 1.20947758e+16
 1.20947758e+16 1.20947758e+16 1.20947758e+16 1.20947758e+16
 1.20947758e+16 1.20947758e+16 1.20947758e+16 1.20947758e+16
 1.20947758e+16 1.20947758e+16 1.20947758e+16 1.20947758e+16
 1.20947758e+16 1.20947758e+16 1.20947758e+16 1.20947758e+16
 1.20947758e+16 1.20947758e+16 1.20947758e+16 1.20947758e+16
 1.20947758e+16 1.20947758e+16 1.20947758e+16 1.20947758e+16
 1.20947758e+16 1.20947758e+16 1.20947758e+16 1.20947758e+16
 1.20947758e+16 1.20947758e+16 1.20947758e+16 1.20947758e+16
 1.20947758e+16 1.20947758e+16 1.20947758e+16 1.20947758e+16
 1.20947758e+16 1.20947758e+16 1.20947758e+16 1.20947758e+16
 1.20947758e+16 1.20947758e+16 1.20947758e+16 1.20947758e+16
 1.20947758e+16 1.20947758e+16 1.20947758e+16 1.20947758e+16
 1.20947758e+16

In [12]:
# Test RQIAPP with 1e-8
result = RQIAPP(M, mu, 1e-8, 100)
print('Eigenvalue:', result[0])
print('Eigenvector:', result[1])
print('Iterations:', result[2])
print('Condition Numbers:', result[3])
print('Condition Numbers for C:', result[4])
print('Desired Condition Numbers:', result[5])

Eigenvalue: (49.838266428417995+0j)
Eigenvector: [0.09550871+0.j 0.10338972+0.j 0.09767328+0.j 0.09968594+0.j
 0.10344816+0.j 0.10123636+0.j 0.09808322+0.j 0.10422956+0.j
 0.10085886+0.j 0.09693695+0.j 0.10004665+0.j 0.09993394+0.j
 0.10008715+0.j 0.09407992+0.j 0.10323378+0.j 0.09513031+0.j
 0.09957521+0.j 0.09707391+0.j 0.09453099+0.j 0.10903108+0.j
 0.0890976 +0.j 0.10702368+0.j 0.10955025+0.j 0.09263746+0.j
 0.11312551+0.j 0.09863057+0.j 0.10169131+0.j 0.1039357 +0.j
 0.08732361+0.j 0.09470451+0.j 0.09873801+0.j 0.10854135+0.j
 0.10494186+0.j 0.10126689+0.j 0.10181169+0.j 0.09940484+0.j
 0.09636202+0.j 0.09995023+0.j 0.09126765+0.j 0.1141403 +0.j
 0.10043883+0.j 0.10341103+0.j 0.09581322+0.j 0.09970896+0.j
 0.09818492+0.j 0.09816775+0.j 0.09204125+0.j 0.10046363+0.j
 0.10267228+0.j 0.09303735+0.j 0.10462168+0.j 0.10536937+0.j
 0.10263202+0.j 0.10174593+0.j 0.10459266+0.j 0.09712412+0.j
 0.09947651+0.j 0.091799  +0.j 0.10769462+0.j 0.10912235+0.j
 0.09876486+0.j 0.10464538+0.j 0.096

In [13]:
# Print actual eigenvalue/eigenvector
print(vals[0], vecs[:,0])

(49.83826642841809+0j) [0.09550871+0.j 0.10338972+0.j 0.09767328+0.j 0.09968594+0.j
 0.10344816+0.j 0.10123636+0.j 0.09808322+0.j 0.10422956+0.j
 0.10085886+0.j 0.09693695+0.j 0.10004665+0.j 0.09993394+0.j
 0.10008715+0.j 0.09407992+0.j 0.10323378+0.j 0.09513031+0.j
 0.09957521+0.j 0.09707391+0.j 0.09453099+0.j 0.10903108+0.j
 0.0890976 +0.j 0.10702368+0.j 0.10955025+0.j 0.09263746+0.j
 0.11312551+0.j 0.09863057+0.j 0.10169131+0.j 0.1039357 +0.j
 0.08732361+0.j 0.09470451+0.j 0.09873801+0.j 0.10854135+0.j
 0.10494186+0.j 0.10126689+0.j 0.10181169+0.j 0.09940484+0.j
 0.09636202+0.j 0.09995023+0.j 0.09126765+0.j 0.1141403 +0.j
 0.10043883+0.j 0.10341103+0.j 0.09581322+0.j 0.09970896+0.j
 0.09818492+0.j 0.09816775+0.j 0.09204125+0.j 0.10046363+0.j
 0.10267228+0.j 0.09303735+0.j 0.10462168+0.j 0.10536937+0.j
 0.10263202+0.j 0.10174593+0.j 0.10459266+0.j 0.09712412+0.j
 0.09947651+0.j 0.091799  +0.j 0.10769462+0.j 0.10912235+0.j
 0.09876486+0.j 0.10464538+0.j 0.09622094+0.j 0.09908714+0.j
 

In [14]:
# Function to run n rounds of RQI and RQIAPP
def runRQI(M, mu, ep, it, n):
  # Save number of iterations to arrays
  its = np.empty(0)
  itsAPP = np.empty(0)

  # Save condition numbers to arrays
  conds = np.empty(0)
  condsAPP = np.empty(0)

  # Save last condition number of each iteration to arrays
  lastConds = np.empty(0)
  lastCondsAPP = np.empty(0)

  for i in range(n):
    RQIData = RQI(M, mu, ep, it)
    RQIAPPData = RQIAPP(M, mu, ep, it)

    # Save number of iterations required to the appropriate array
    its = np.append(its, RQIData[2])
    itsAPP = np.append(itsAPP, RQIAPPData[2])

    # Save condition numbers to the appropriate array
    conds = np.append(conds, RQIData[3])
    condsAPP = np.append(condsAPP, RQIAPPData[4])

    # Save last condition number to the appropriate array
    lastConds = np.append(conds, RQIData[3][-1])
    lastCondsAPP = np.append(condsAPP, RQIAPPData[4][-1])

  # Print iterations required each round
  print('Iterations required each round for RQI:\n', its)
  print('\nIterations required each round for RQIAPP:\n', itsAPP)

  # Get average iterations required by RQI
  index = np.argwhere(its==it)
  its = np.delete(its, index)
  if its.size==0:
    itsAvg = 0
  else:
    itsAvg = np.average(its)

  # Get average condition number for RQI
  condsAvg = np.average(conds)

  # Get average final condition number for RQI
  lastCondsAvg = np.average(lastConds)

  # Get average iterations required by RQIAPP
  index = np.argwhere(itsAPP==it)
  itsAPP = np.delete(itsAPP, index)
  if itsAPP.size==0:
    itsAPPAvg = 0
  else:
    itsAPPAvg = np.average(itsAPP)

  # Get average condition number for RQIAPP
  condsAPPAvg = np.average(condsAPP)

  # Get average final condition number for RQI
  lastCondsAPPAvg = np.average(lastCondsAPP)

  # Print average iterations required and number of errors
  print('\nAverage iterations required for RQI:\n', itsAvg)
  print('\nAveragee condition number for RQI:\n', condsAvg)
  print('\nAveragee final condition number for RQI:\n', lastCondsAvg)
  print('\nPercentage of times RQI converged:\n', str(its.size/n*100) + '%')
  print('\nAverage iterations required for RQIAPP:\n', itsAPPAvg)
  print('\nAverage condition number for RQIAPP:\n', condsAPPAvg)
  print('\nAverage final condition number for RQIAPP:\n', lastCondsAPPAvg)
  print('\nPercentage of times RQIAPP converged:\n', str(itsAPP.size/n*100) + '%')

In [15]:
# Run 100 rounds with 1e-4
runRQI(M, mu, 1e-4, 20, 100)

Iterations required each round for RQI:
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2.]

Iterations required each round for RQIAPP:
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2.]

Average iterations required for RQI:
 2.0

Averagee condition number for RQI:
 75845.94585088894

Averagee final condition number for RQI:
 75562.65203397314

Percentage of times RQI converged:
 100.0%

Average iterations required for RQIAPP:
 2.0

Average condition number for RQIAPP:
 1.991102340882

In [16]:
# Run 100 rounds with 1e-6
runRQI(M, mu, 1e-6, 20, 100)

Iterations required each round for RQI:
 [ 3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
  3.  3. 20.  3.  3.  3.  3. 20.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
  3.  3.  3.  3.  3.  3. 20.  3.  3.  3.]

Iterations required each round for RQIAPP:
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2.]

Average iterations required for RQI:
 3.0

Averagee condition number for RQI:
 1690811255678642.8

Averagee final condition number for RQI:
 1686007818561170.2

Percentage of times RQI converged:
 97.

In [17]:
# Run 100 rounds with 1e-8
runRQI(M, mu, 1e-8, 20, 100)

Iterations required each round for RQI:
 [20.  3. 20.  3.  3. 20. 20.  3. 20. 20. 20. 20. 20. 20. 20. 20.  3. 20.
 20.  3. 20. 20. 20. 20. 20. 20.  3. 20.  3. 20.  3. 20.  3. 20. 20. 20.
 20. 20. 20. 20. 20.  3. 20.  3. 20. 20.  3. 20.  3.  3. 20. 20. 20. 20.
 20. 20. 20. 20.  3. 20. 20. 20. 20. 20. 20.  3. 20.  3.  3. 20. 20. 20.
 20. 20. 20. 20.  3. 20. 20. 20. 20.  3. 20.  3. 20. 20.  3. 20.  3. 20.
 20.  3. 20. 20.  3. 20.  3.  3. 20. 20.]

Iterations required each round for RQIAPP:
 [3. 2. 2. 3. 3. 3. 3. 3. 3. 3. 3. 2. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 2. 3. 3. 3. 3. 3. 3. 2. 3. 3. 3. 3. 3. 2. 3. 3. 3. 2. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 2. 3. 2. 3. 3. 3. 2. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 2. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 2. 2. 3. 3.
 3. 3. 3. 3.]

Average iterations required for RQI:
 3.0

Averagee condition number for RQI:
 9237526976128248.0

Averagee final condition number for RQI:
 9239400581913976.0

Percentage of times RQI converged:
 28.

In [18]:
# Run 100 rounds with 1e-10
runRQI(M, mu, 1e-10, 20, 100)

Iterations required each round for RQI:
 [20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.
 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.
 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.
 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.
 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.
 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]

Iterations required each round for RQIAPP:
 [3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 2. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3.]

Average iterations required for RQI:
 0

Averagee condition number for RQI:
 9732951290402578.0

Averagee final condition number for RQI:
 9734131612496014.0

Percentage of times RQI converged:
 0.0%
